In [6]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tqdm.keras import TqdmCallback
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
base_dir = 'C:/Users/user/Downloads/data/pre_image'

In [7]:
# 데이터 전처리
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# 모델 구성 함수
def build_model(model_name, input_size, num_classes=4):
    if model_name == 'basic_cnn':
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(input_size, input_size, 3)),
            MaxPooling2D(2, 2),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D(2, 2),
            Conv2D(128, (3, 3), activation='relu'),
            MaxPooling2D(2, 2),
            Conv2D(256, (3, 3), activation='relu'),  # Conv2D 256 필터 추가
            MaxPooling2D(2, 2),
            Flatten(),
            Dense(512, activation='relu'),
            Dense(num_classes, activation='softmax')
        ])
    elif model_name == 'vgg16':
        base_model = VGG16(input_shape=(input_size, input_size, 3), include_top=False, weights='imagenet')
        base_model.trainable = False
        model = Sequential([
            base_model,
            Flatten(),
            Dense(512, activation='relu'),
            Dropout(0.5),
            Dense(num_classes, activation='softmax')
        ])
    elif model_name == 'resnet50': 
        base_model = ResNet50(input_shape=(input_size, input_size, 3), include_top=False, weights='imagenet')
        base_model.trainable = False
        model = Sequential([
            base_model,
            Flatten(),
            Dense(512, activation='relu'),
            Dropout(0.5),
            Dense(num_classes, activation='softmax')
        ])
    # 모델 컴파일
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 모델, target size, batch size, epochs 설정
models_info = [
    ('basic_cnn', [120, 150]),  # target size 수정: 60 삭제, 150 추가
    ('vgg16', [120, 150]),
    ('resnet50', [224]),
]
batch_size = 128
epochs = 20

# Early stopping 콜백 설정
early_stop = EarlyStopping(monitor='val_loss', patience=3)  # 3회 이상 개선이 없으면 학습 중단

# 성능 저장을 위한 딕셔너리
performance_metrics = {}

# 각 모델별로 훈련 및 성능 평가
for model_info in models_info:
    model_name, sizes = model_info
    for size in sizes:
        print(f"Training {model_name} with size {size}")

        train_generator = train_datagen.flow_from_directory(
            base_dir,
            target_size=(size, size),
            batch_size=batch_size,
            class_mode='categorical',
            subset='training')
        
        validation_generator = train_datagen.flow_from_directory(
            base_dir,
            target_size=(size, size),
            batch_size=batch_size,
            class_mode='categorical',
            subset='validation')
        
        model = build_model(model_name, size)
        model.summary()  # 모델 구조 출력

        # 모델 학습
        history = model.fit(
            train_generator,
            steps_per_epoch=train_generator.samples // batch_size,
            validation_data=validation_generator,
            validation_steps=validation_generator.samples // batch_size,
            epochs=epochs,
            callbacks=[TqdmCallback(verbose=1), early_stop],  # EarlyStopping 콜백 추가
            verbose=0  # tqdm을 사용하기 때문에 기본 출력은 끕니다.
        )

Training basic_cnn with size 120
Found 12383 images belonging to 4 classes.
Found 3093 images belonging to 4 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 59, 59, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 28, 28, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 128)       73856     
    

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Training basic_cnn with size 150
Found 12383 images belonging to 4 classes.
Found 3093 images belonging to 4 classes.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 34, 34, 128)       73856     
  

0epoch [00:00, ?epoch/s]

...

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Training resnet50 with size 224
Found 12383 images belonging to 4 classes.
Found 3093 images belonging to 4 classes.
94765736/94765736 [==============================] - 4s 0us/step
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_4 (Flatten)         (None, 100352)            0         
                                                                 
 dense_8 (Dense)             (None, 512)               51380736  
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 4)                 2052      
                                                                 
Tota

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

In [10]:
import os
import json

# 모델 및 히스토리 저장 디렉토리 생성
save_dir = "saved_models"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 각 모델별로 훈련 및 성능 평가
for model_info in models_info:
    model_name, sizes = model_info
    for size in sizes:
        print(f"Training {model_name} with size {size}")

        # 모델 저장
        model_save_path = os.path.join(save_dir, f"{model_name}_{size}.h5")
        model.save(model_save_path)
        print(f"Model saved at {model_save_path}")

        # 히스토리 저장
        history_dict = history.history
        history_save_path = os.path.join(save_dir, f"{model_name}_{size}_history.json")
        with open(history_save_path, 'w') as file:
            json.dump(history_dict, file)
        print(f"Training history saved at {history_save_path}")


Training basic_cnn with size 120


c:\Users\user\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved at saved_models\basic_cnn_120.h5
Training history saved at saved_models\basic_cnn_120_history.json
Training basic_cnn with size 150
Model saved at saved_models\basic_cnn_150.h5
Training history saved at saved_models\basic_cnn_150_history.json
Training vgg16 with size 120
Model saved at saved_models\vgg16_120.h5
Training history saved at saved_models\vgg16_120_history.json
Training vgg16 with size 150
Model saved at saved_models\vgg16_150.h5
Training history saved at saved_models\vgg16_150_history.json
Training resnet50 with size 224
Model saved at saved_models\resnet50_224.h5
Training history saved at saved_models\resnet50_224_history.json
